
The points distribution for this case is as follows:

Read the data set
Drop the columns which are unique for all users like IDs (2.5 points)
Distinguish the feature and target set (2.5 points)
Divide the data set into training and test sets ( 2.5 points)
Normalize the train and test data (5 points)
Initialize & build the model (10 points)
Predict the results using 0.5 as a threshold (5 points)
Print the Accuracy score and confusion matrix (2.5 points)

In [152]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

In [153]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Neural Networks')

In [155]:
df = pd.read_csv("bank.csv")

In [156]:
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


2. Drop the columns which are unique for all users like IDs

In [157]:
df['CreditScore'].value_counts()

850    233
678     63
655     54
705     53
667     53
      ... 
419      1
417      1
373      1
365      1
401      1
Name: CreditScore, Length: 460, dtype: int64

In [158]:
df['CustomerId'].value_counts()

15812607    1
15741078    1
15635776    1
15740223    1
15738174    1
           ..
15743714    1
15639265    1
15641312    1
15684319    1
15695872    1
Name: CustomerId, Length: 10000, dtype: int64

In [159]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [160]:
#RowNumber #CustomerId and #Surname are unique hence dropping it
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


3. Distinguish the feature and target set

In [162]:
X = df.iloc[:,0:10].values 
y = df.iloc[:,10].values # customers who left

In [163]:
#applying categorical encoding for country
print(X[:8,1])
label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany']
[0 2 0 0 2 2 0 1]


In [164]:
# applying categorical encoder on gender
print(X[:6,2])
label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male']
[0 0 0 0 0 1]


In [165]:
#applying one hot encoding


columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])],     remainder='passthrough')
X=np.array(columnTransformer.fit_transform(X),dtype=np.str)


In [166]:
X.shape

(10000, 12)

In [167]:
X

array([['1.0', '0.0', '0.0', ..., '1', '1', '101348.88'],
       ['0.0', '0.0', '1.0', ..., '0', '1', '112542.58'],
       ['1.0', '0.0', '0.0', ..., '1', '0', '113931.57'],
       ...,
       ['1.0', '0.0', '0.0', ..., '0', '1', '42085.58'],
       ['0.0', '1.0', '0.0', ..., '1', '0', '92888.52'],
       ['1.0', '0.0', '0.0', ..., '1', '0', '38190.78']], dtype='<U9')

In [168]:
X = X[:,1:] # Got rid of Spain as a dimension.

Divide the data set into Train and test sets

In [169]:
# Splitting the dataset into the Training and Testing set.

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

Normalize the train and test data

In [170]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Initialize & build the model

In [171]:
# Initializing
classifier = Sequential()

In [172]:
# nodes = 6 is average of input and output layers, in this case 6.
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [173]:
# 1st hidden layer
classifier.add(Dense(6, activation='sigmoid', kernel_initializer='uniform'))

In [174]:
# output dimension is 1
#there are probability outcomes hence we use sigmoid
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform')) 

In [175]:
# Create optimizer with default learning rate and compile

classifier.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

In [176]:
classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 6)                 72        
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [177]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2263 - accuracy: 0.7406 - val_loss: 0.1832 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1772 - accuracy: 0.7997 - val_loss: 0.1684 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1666 - accuracy: 0.7984 - val_loss: 0.1642 - val_accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1656 - accuracy: 0.7937 - val_loss: 0.1627 - val_accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1655 - accuracy: 0.7919 - val_loss: 0.1620 - val_accuracy: 0.7975
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1619 - accuracy: 0.7976 - val_loss: 0.1618 - val_accuracy: 0.7975
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1625 - accuracy: 0.7961 - val_loss: 0.1616 - val_accuracy: 0.7975

Predict the results using 0.5 as a threshold

In [178]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.20438898]
 [0.20436683]
 [0.20329225]
 ...
 [0.20407787]
 [0.20399702]
 [0.20425785]]


In [179]:
 # cutoff value 0.5 to show whether customer will leave or not
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


Print the Accuracy score and confusion matrix

In [180]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false.
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

[[1595    0]
 [ 405    0]]


In [181]:
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print (accuracy_model1, '% of testing data was classified correctly')

79.75 % of testing data was classified correctly


Optimize the model

Some important parameters to look out for while optimizing neural networks are:

-Type of architecture

-Number of Layers

-Number of Neurons in a layer

-Regularization parameters

-Learning Rate

-Type of optimization / backpropagation technique to use

-Dropout rate

-Weight sharing

Number of Layers:
We will keep it similar to the above model so that we can compare the accuracy. 1 hidden layer.

Activation:
input layer: relu becasue we are in an input layer. uses the ReLu activation function for ϕ output layer: sigmoid becasue we are in an output layer. uses the Sigmoid activation function for ϕ . This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.

Type of optimization / backpropagation technique to use:
We will use Adam. Adam is a very efficeint variation of Stochastic Gradient Descent. For Adam and its variant, learning rate or the decay rate does not really matter too much.

Learning Rate:
default learning rate 0.001.

Number of Neurons in a layer:
We will keep it 6 as per our initial calculation above.

Weight sharing / kernel_initializer:
uniform the distribution with which we randomly initialize weights for the nodes in this layer.

Loss:
loss: binary_crossentropy This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is Binary, it is binary_crossentropy. If Categorical, then it is called categorical_crossentropy

Rebuilding the model using these optimised parameters

In [191]:
# Initializing 
classifier = Sequential()
# add the input layer with input dimension AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [192]:
#output layer
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform'))

In [193]:
# Sigmoid to get probabilities
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform'))

In [194]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [195]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6497 - accuracy: 0.7861 - val_loss: 0.4445 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4358 - accuracy: 0.8004 - val_loss: 0.4321 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4335 - accuracy: 0.7916 - val_loss: 0.4269 - val_accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4293 - accuracy: 0.7952 - val_loss: 0.4238 - val_accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4138 - accuracy: 0.8027 - val_loss: 0.4219 - val_accuracy: 0.7975
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4217 - accuracy: 0.7970 - val_loss: 0.4188 - val_accuracy: 0.7975
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4148 - accuracy: 0.8014 - val_loss: 0.4164 - val_accuracy: 0.8200

Predict the results using 0.5 as a threshold

In [196]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.19567174]
 [0.33741406]
 [0.21178603]
 ...
 [0.16165677]
 [0.1499066 ]
 [0.08599672]]


In [197]:

y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]



Print the Accuracy score and confusion matrix

In [198]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)

[[1546   49]
 [ 261  144]]


In [199]:
accuracy_model2 = ((cm2[0][0]+cm2[1][1])*100)/(cm2[0][0]+cm2[1][1]+cm2[0][1]+cm2[1][0])
print (accuracy_model2, '% of testing data was classified correctly')

84.5 % of testing data was classified correctly


So model accuracy is improved